In [394]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup, SoupStrainer
import re
from urllib.parse import urljoin

Scraping gamelogs for every team from seasons 2014-Present and saving each to a csv file 

In [395]:
years = list(range(2015,2024))

In [396]:
url_start = 'https://www.basketball-reference.com/teams/HOU/{}/gamelog/'

In [397]:
for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open('hou_gamelog/{}.html'.format(year), 'w+', encoding = 'utf-8') as nba:
        nba.write(data.text)

FileNotFoundError: [Errno 2] No such file or directory: 'hou_gamelog/2015.html'

In [ ]:
#open 1 page example

with open('hou_gamelog/2017.html', encoding = 'utf-8') as nba:
    page = nba.read()

In [72]:
#soup = bs(page, 'html.parser')

In [73]:
#soup.find('tr', class_= 'over_header').decompose() 

In [74]:
#bos_gmlog_table = soup.find(id ='tgl_basic')

In [75]:
#bos_gmlog_17 = pd.read_html(str(bos_gmlog_table))[0]

In [76]:
#bos_gmlog_17

In [253]:
dfs = []
for year in years:
    with open('hou_gamelog/{}.html'.format(year), encoding = 'utf-8') as nba:
        page = nba.read()
        soup = bs(page, 'html.parser')
        soup.find('tr', class_= 'over_header').decompose() 
        soup.find('tr', class_= 'over_header thead').decompose()
        hou_gmlog_table = soup.find(id ='tgl_basic')
        hou_gmlogs_14_23 = pd.read_html(str(hou_gmlog_table))[0]
    
    dfs.append(hou_gmlogs_14_23)

In [255]:
hou = pd.concat(dfs)

In [256]:
hou = hou.rename(columns = {'Unnamed: 3': 'A/H',
                                        'Tm': 'Tm_PTS',
                                         'Opp.1': 'Opp_PTS',
                                         '3PA': 'Three_FGA',
                                         '3PA.1': 'OThree_FGA'})

In [257]:
hou['A/H'] = hou['A/H'].replace(np.nan, 'H')
hou = hou.replace({'@':'A'})

In [259]:
hou.to_csv('hou.csv')

In [390]:
URL = 'https://www.basketball-reference.com/teams/BOS/2023.html'

r = requests.get(URL)

In [391]:
soup = bs(r.text, 'html.parser')

In [462]:
for href in soup.find_all('a', href=re.compile(r'.+gamelog.+')):
    playerid = href['href'][11:-14]
    gmlog = re.findall(r'\/players\/[a-z]\/(.+)\/gamelog\/2023\/', playerid)
    print(playerid)



tatumja01
brownja02
smartma01
horfoal01
willigr01
whitede01
brogdma01
williro04
hausesa01
griffbl01
kornelu01
pritcpa01
kabenmf01
vonleno01
jacksju01
davisjd01
tatumja01
brownja02
smartma01
willigr01
whitede01
horfoal01
brogdma01
hausesa01
kornelu01
pritcpa01
griffbl01
williro04
vonleno01
jacksju01
davisjd01
kabenmf01
tatumja01
brownja02
smartma01
willigr01
whitede01
horfoal01
brogdma01
hausesa01
kornelu01
pritcpa01
griffbl01
williro04
vonleno01
jacksju01
davisjd01
kabenmf01



In [341]:
plyrid = gmlog
letter = gmlog[0]
year = 2023

IndexError: list index out of range

In [193]:
pd.read_html(f'https://www.basketball-reference.com/players/{letter}/{g}/gamelog/{year}')[-1] 

NameError: name 'letter' is not defined

Cleaning and creating East/West/League gamelog dataframes. Adding datetime difference columns to League dataframe.  

In [175]:
nba_east_gamelog_14_23 = pd.concat(
    map(pd.read_csv,['nba_east/atl.csv', 'nba_east/bos.csv', 'nba_east/brk.csv', 'nba_east/chi.csv',
                    'nba_east/cho.csv', 'nba_east/cle.csv', 'nba_east/det.csv', 'nba_east/ind.csv',
                    'nba_east/mia.csv', 'nba_east/mil.csv', 'nba_east/nyk.csv', 'nba_east/orl.csv',
                    'nba_east/phi.csv', 'nba_east/tor.csv', 'nba_east/was.csv']), ignore_index = True)

In [176]:
nba_east_gamelog_14_23 

,Unnamed: 0,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,0,ATL,1,1,2014-10-29,A,TOR,L,102,109,...,27,33,.818,16,48,26,13,9,9,22
1,1,ATL,2,2,2014-11-01,H,IND,W,102,92,...,18,21,.857,11,44,25,5,5,18,26
2,2,ATL,3,3,2014-11-05,A,SAS,L,92,94,...,27,38,.711,11,50,25,7,9,19,15
3,3,ATL,4,4,2014-11-07,A,CHO,L,119,122,...,20,27,.741,11,51,31,6,7,19,30
4,4,ATL,5,5,2014-11-08,H,NYK,W,103,96,...,8,11,.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10878,723,WAS,35,35,2022-12-27,H,PHI,W,116,111,...,27,31,.871,10,39,21,8,10,15,24
10879,724,WAS,36,36,2022-12-28,H,PHO,W,127,102,...,11,16,.688,11,30,25,7,4,11,21
10880,725,WAS,37,37,2022-12-30,A,ORL,W,119,100,...,11,16,.688,8,35,24,8,2,11,18
10881,726,WAS,38,38,2023-01-01,A,MIL,W,118,95,...,14,15,.933,14,44,22,11,3,16,15


In [174]:
nba_east_gamelog_14_23.to_csv('nba_east_gamelog_14_23.csv') 

In [168]:
nba_west_gamelog_14_23 = pd.concat(
    map(pd.read_csv,['nba_west/dal.csv', 'nba_west/den.csv', 'nba_west/gsw.csv', 'nba_west/hou.csv',
                     'nba_west/lac.csv', 'nba_west/lal.csv', 'nba_west/mem.csv', 'nba_west/mem.csv',
                     'nba_west/min.csv', 'nba_west/nop.csv', 'nba_west/okc.csv', 'nba_west/pho.csv',
                     'nba_west/por.csv', 'nba_west/sac.csv', 'nba_west/sas.csv', 'nba_west/uta.csv']), ignore_index = True)

In [177]:
nba_west_gamelog_14_23

,Unnamed: 0,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,0,DAL,1,1,2014-10-28,A,SAS,L,100,101,...,13,16,.813,9,38,23,5,3,20,20
1,1,DAL,2,2,2014-10-30,H,UTA,W,120,102,...,16,18,.889,15,40,22,6,0,16,21
2,2,DAL,3,3,2014-11-01,A,NOP,W,109,104,...,21,28,.750,14,47,24,8,4,13,19
3,3,DAL,4,4,2014-11-03,H,BOS,W,118,113,...,12,16,.750,22,49,29,9,6,17,21
4,4,DAL,5,5,2014-11-06,A,POR,L,87,108,...,16,19,.842,12,53,25,3,6,17,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11629,727,UTA,39,39,2022-12-31,H,MIA,L,123,126,...,23,30,.767,10,37,32,10,1,12,32
11630,728,UTA,40,40,2023-01-03,H,SAC,L,115,117,...,16,18,.889,7,40,23,7,4,17,26
11631,729,UTA,NaN,NaN,NaN,H,NaN,NaN,NaN,NaN,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
11632,730,UTA,Rk,G,Date,H,Opp,W/L,Tm,Opp,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF


In [6]:
nba_west_gamelog_14_23.to_csv('nba_west_gamelog_14_23.csv')

In [106]:
nba_league_gmlg_14_23 = pd.concat(
    map(pd.read_csv,['nba_east/nba_east_gamelog_14_23.csv', 'nba_west/nba_west_gamelog_14_23.csv']), ignore_index = True)

In [167]:
nba_east_gamelog_14_23

,Unnamed: 0,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,0,ATL,1,1,2014-10-29,A,TOR,L,102,109,...,27,33,.818,16,48,26,13,9,9,22
1,1,ATL,2,2,2014-11-01,H,IND,W,102,92,...,18,21,.857,11,44,25,5,5,18,26
2,2,ATL,3,3,2014-11-05,A,SAS,L,92,94,...,27,38,.711,11,50,25,7,9,19,15
3,3,ATL,4,4,2014-11-07,A,CHO,L,119,122,...,20,27,.741,11,51,31,6,7,19,30
4,4,ATL,5,5,2014-11-08,H,NYK,W,103,96,...,8,11,.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10878,723,WAS,35,35,2022-12-27,H,PHI,W,116,111,...,27,31,.871,10,39,21,8,10,15,24
10879,724,WAS,36,36,2022-12-28,H,PHO,W,127,102,...,11,16,.688,11,30,25,7,4,11,21
10880,725,WAS,37,37,2022-12-30,A,ORL,W,119,100,...,11,16,.688,8,35,24,8,2,11,18
10881,726,WAS,38,38,2023-01-01,A,MIL,W,118,95,...,14,15,.933,14,44,22,11,3,16,15


In [132]:
nba_league_gmlg_14_23 = nba_league_gmlg_14_23[nba_league_gmlg_14_23.Date != 'Date']
nba_league_gmlg_14_23  = nba_league_gmlg_14_23.dropna(axis=0)
nba_league_gmlg_14_23 = nba_league_gmlg_14_23.drop(['Unnamed: 0.1'], axis =1)


KeyError: "['Unnamed: 0.1'] not found in axis"

In [133]:
nba_league_gmlg_14_23

,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,FG,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,ATL,1,1,2014-10-29,A,TOR,L,102,109,40,...,27,33,.818,16,48,26,13,9,9,22
1,ATL,2,2,2014-11-01,H,IND,W,102,92,35,...,18,21,.857,11,44,25,5,5,18,26
2,ATL,3,3,2014-11-05,A,SAS,L,92,94,38,...,27,38,.711,11,50,25,7,9,19,15
3,ATL,4,4,2014-11-07,A,CHO,L,119,122,43,...,20,27,.741,11,51,31,6,7,19,30
4,ATL,5,5,2014-11-08,H,NYK,W,103,96,33,...,8,11,.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22510,UTA,37,37,2022-12-28,A,GSW,L,107,112,37,...,22,27,.815,10,51,22,6,3,12,26
22511,UTA,38,38,2022-12-30,A,SAC,L,125,126,46,...,14,19,.737,10,38,32,5,3,15,21
22512,UTA,39,39,2022-12-31,H,MIA,L,123,126,35,...,23,30,.767,10,37,32,10,1,12,32
22513,UTA,40,40,2023-01-03,H,SAC,L,115,117,38,...,16,18,.889,7,40,23,7,4,17,26


In [152]:
nba_league_gmlg_14_23['Date'] = pd.to_datetime(nba_league_gmlg_14_23['Date'], format='%Y-%m-%d')
nba_league_gmlg_14_23['Days'] = nba_league_gmlg_14_23.groupby('Tm')['Date'].diff().dt.days
nba_league_gmlg_14_23['Hours'] = nba_league_gmlg_14_23.groupby('Tm')['Date'].diff().dt.days.multiply(24)
nba_league_gmlg_14_23['Minutes'] = nba_league_gmlg_14_23.groupby('Tm')['Date'].diff().dt.days.multiply(60)

In [157]:
nba_league_gmlg_14_23 = nba_league_gmlg_14_23.fillna(0)

In [162]:
nba_league_gmlg_14_23 = nba_league_gmlg_14_23.rename(columns={'3P':'TFG','Three_FGA':'TFGA','3P%':'TP%',
                                      'FG.1':'Opp_FG', 'FGA.1': 'Opp_FGA',
                                      'FG%.1':'Opp_FG%', '3P.1':'Opp_TFG',
                                      'OThree_FGA':'Opp_TFGA', '3P%.1':'Opp_TP%',
                                      'FT.1':'Opp_FT', 'FTA.1':'Opp_FTA',
                                      'FT%.1':'Opp_FT%', 'ORB.1':'Opp_ORB',
                                      'TRB.1':'Opp_TRB', 'AST.1':'Opp_AST',
                                      'STL.1':'Opp_STL','BLK.1':'Opp_BLK', 
                                      'TOV.1':'Opp_TOV','PF.1':'Opp_PF'})
                            
                                      

In [163]:
nba_league_gmlg_14_23

,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,FG,...,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Rest,Days,Hours,Minutes
0,ATL,1,1,2014-10-29,A,TOR,L,102,109,40,...,48,26,13,9,9,22,0.0,0.0,0.0,0.0
1,ATL,2,2,2014-11-01,H,IND,W,102,92,35,...,44,25,5,5,18,26,3.0,3.0,72.0,180.0
2,ATL,3,3,2014-11-05,A,SAS,L,92,94,38,...,50,25,7,9,19,15,4.0,4.0,96.0,240.0
3,ATL,4,4,2014-11-07,A,CHO,L,119,122,43,...,51,31,6,7,19,30,2.0,2.0,48.0,120.0
4,ATL,5,5,2014-11-08,H,NYK,W,103,96,33,...,44,26,2,6,15,29,1.0,1.0,24.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22510,UTA,37,37,2022-12-28,A,GSW,L,107,112,37,...,51,22,6,3,12,26,2.0,2.0,48.0,120.0
22511,UTA,38,38,2022-12-30,A,SAC,L,125,126,46,...,38,32,5,3,15,21,2.0,2.0,48.0,120.0
22512,UTA,39,39,2022-12-31,H,MIA,L,123,126,35,...,37,32,10,1,12,32,1.0,1.0,24.0,60.0
22513,UTA,40,40,2023-01-03,H,SAC,L,115,117,38,...,40,23,7,4,17,26,3.0,3.0,72.0,180.0


In [164]:
nba_league_gmlg_14_23.to_csv('nba_league_gmlg_14_23.csv')
